In [ ]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic


# Sett opp OpenCageGeocode med API-nøkkelen din
api_key = 'e9a6e9b1b9be44fab4720c2c96ae9c48'  # Sett inn din OpenCage API-nøkkel
geolocator = OpenCageGeocode(api_key)

# Henter inn to lokasjoner fra brukeren
location_input1 = input("Enter the first location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")
location_input2 = input("Enter the second location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")

# Geokoder lokasjonene for å få koordinatene
location1 = geolocator.geocode(location_input1)
location2 = geolocator.geocode(location_input2)

# Sjekker om geokodingen var vellykket
if location1 and location2:
    # Henter ut koordinatene
    location_coords1 = (location1[0]['geometry']['lat'], location1[0]['geometry']['lng'])
    location_coords2 = (location2[0]['geometry']['lat'], location2[0]['geometry']['lng'])
else:
    raise ValueError("Could not find coordinates for one or both locations. Please check the input.")

# Vår klient-id for å hente data fra Frost API
client_id = '2e243d34-57bc-42b4-8095-239991af5353'

# Definerer endepunkt og parametere for stasjoner
stations_endpoint = 'https://frost.met.no/sources/v0.jsonld'
stations_parameters = {
    'types': 'SensorSystem'
}

# Henter ut HTTP-responsen og lagrer den som JSON
r = requests.get(stations_endpoint, stations_parameters, auth=(client_id, ''))
stations_json = r.json()

# Henter ut stasjonsinformasjonen
stations = stations_json['data']

# Lager et dictionary med stasjons-ID som nøkkel og koordinater som verdi
weather_stations_coords = {station['id']: (station['geometry']['coordinates'][1], station['geometry']['coordinates'][0]) for station in stations if 'geometry' in station}

# Regner ut avstanden til værstasjonene fra den første lokasjonen
distances1 = {station: geodesic(location_coords1, coords).km for station, coords in weather_stations_coords.items()}
# Regner ut avstanden til værstasjonene fra den andre lokasjonen
distances2 = {station: geodesic(location_coords2, coords).km for station, coords in weather_stations_coords.items()}

# Sorterer værstasjonene etter avstand til lokasjonene
sorted_stations1 = sorted(distances1, key=distances1.get)
sorted_stations2 = sorted(distances2, key=distances2.get)

# Henter ut dagens dato og datoen 90 dager tilbake
now = datetime.now()
offset = now - timedelta(days=90)
current_date = now.strftime("%Y-%m-%d")
historic_date = offset.strftime("%Y-%m-%d")

# Definerer endepunkt og parametere for observasjoner
observations_endpoint = 'https://frost.met.no/observations/v0.jsonld'

def fetch_data(stations, max_attempts=5):
    """
    Fetch data from the nearest weather stations.
    Tries up to max_attempts stations and returns the data from the first successful attempt.
    """
    for attempt in range(min(max_attempts, len(stations))):
        parameters = {
            'sources': stations[attempt],
            'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
            'referencetime': f'{historic_date}/{current_date}',
        }
        r = requests.get(observations_endpoint, parameters, auth=(client_id, ''))
        json = r.json()

        if r.status_code == 200 and 'data' in json:
            data = json['data']
            if data:
                print(f'Data retrieved from {stations[attempt]}!')
                return data, stations[attempt]
        else:
            print(f'Error retrieving data from {stations[attempt]}. Trying next station...')

    # Hvis ingen data blir funnet, returneres data fra den første stasjonen
    parameters = {
        'sources': stations[0],
        'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
        'referencetime': f'{historic_date}/{current_date}',
    }
    r = requests.get(observations_endpoint, parameters, auth=(client_id, ''))
    json = r.json()
    data = json['data']
    print(f'No data found from other stations. Using data from {stations[0]}.')
    return data, stations[0]

# Henter data for begge lokasjonene
data1, used_station_id1 = fetch_data(sorted_stations1)
data2, used_station_id2 = fetch_data(sorted_stations2)

# Kartlegging av sourceId til stasjonsnavn
station_names = {station['id']: station['name'] for station in stations if 'name' in station}

important_keys = ['elementId', 'value', 'unit', 'timeOffset', 'timeResolution', 'qualityCode']

# Samler de viktige dataene i DataFrames
df_list1 = []
df_list2 = []

for item in data1:
    row = pd.DataFrame(item['observations'])
    row['referenceTime'] = item['referenceTime']
    row['sourceId'] = item['sourceId']
    df_list1.append(row)

for item in data2:
    row = pd.DataFrame(item['observations'])
    row['referenceTime'] = item['referenceTime']
    row['sourceId'] = item['sourceId']
    df_list2.append(row)

# Effektivt å koble sammen alle DataFrames samtidig
df1 = pd.concat(df_list1, ignore_index=True)
df2 = pd.concat(df_list2, ignore_index=True)

# Plotter dataene
# Separerer dataene etter elementId
temperature_data1 = df1[df1['elementId'] == 'mean(air_temperature P1D)']
precipitation_data1 = df1[df1['elementId'] == 'sum(precipitation_amount P1D)']
wind_speed_data1 = df1[df1['elementId'] == 'mean(wind_speed P1D)']

temperature_data2 = df2[df2['elementId'] == 'mean(air_temperature P1D)']
precipitation_data2 = df2[df2['elementId'] == 'sum(precipitation_amount P1D)']
wind_speed_data2 = df2[df2['elementId'] == 'mean(wind_speed P1D)']

# Konverterer referansetiden til datetime
temperature_data1.loc[:, 'referenceTime'] = pd.to_datetime(temperature_data1['referenceTime'])
precipitation_data1.loc[:, 'referenceTime'] = pd.to_datetime(precipitation_data1['referenceTime'])
wind_speed_data1.loc[:, 'referenceTime'] = pd.to_datetime(wind_speed_data1['referenceTime'])

temperature_data2.loc[:, 'referenceTime'] = pd.to_datetime(temperature_data2['referenceTime'])
precipitation_data2.loc[:, 'referenceTime'] = pd.to_datetime(precipitation_data2['referenceTime'])
wind_speed_data2.loc[:, 'referenceTime'] = pd.to_datetime(wind_speed_data2['referenceTime'])



Data retrieved from SN90450!
Data retrieved from SN82290!


Plotting med Plotly

In [16]:
import plotly.graph_objects as go

# Definer farger for lokasjonene
color_location1 = '#1f77b4'  # Blå for lokasjon 1
color_location2 = '#ff7f0e'  # Oransje for lokasjon 2

# Temperaturfigur
fig_temp = go.Figure()

# Temperaturdata for lokasjon 1
fig_temp.add_trace(go.Scatter(
    x=temperature_data1['referenceTime'],
    y=temperature_data1['value'],
    mode='lines',
    name=f'Temperature at {station_names.get(used_station_id1, "Unknown Station 1")}',
    line=dict(color=color_location1)
))

# Temperaturdata for lokasjon 2
fig_temp.add_trace(go.Scatter(
    x=temperature_data2['referenceTime'],
    y=temperature_data2['value'],
    mode='lines',
    name=f'Temperature at {station_names.get(used_station_id2, "Unknown Station 2")}',
    line=dict(color=color_location2)
))

# Gjennomsnittstemperatur for lokasjon 1
mean_temp1 = temperature_data1['value'].mean()
fig_temp.add_trace(go.Scatter(
    x=temperature_data1['referenceTime'],
    y=[mean_temp1] * len(temperature_data1['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location1),
    name=f'Average Temperature at {station_names.get(used_station_id1, "Unknown Station 1")}'
))

# Gjennomsnittstemperatur for lokasjon 2
mean_temp2 = temperature_data2['value'].mean()
fig_temp.add_trace(go.Scatter(
    x=temperature_data2['referenceTime'],
    y=[mean_temp2] * len(temperature_data2['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location2),
    name=f'Average Temperature at {station_names.get(used_station_id2, "Unknown Station 2")}'
))

# Tilpasser layout for temperatur
fig_temp.update_layout(
    title='Temperatur over tid',
    xaxis_title='Dato',
    yaxis_title='Temperatur (°C)',
    legend_title='Lokasjoner'
)

# Viser temperaturfigur
fig_temp.show()

# Nedbørsfigur
fig_precip = go.Figure()

# Nedbørsdata for lokasjon 1
fig_precip.add_trace(go.Scatter(
    x=precipitation_data1['referenceTime'],
    y=precipitation_data1['value'],
    mode='lines',
    name=f'Precipitation at {station_names.get(used_station_id1, "Unknown Station 1")}',
    line=dict(color=color_location1)
))

# Nedbørsdata for lokasjon 2
fig_precip.add_trace(go.Scatter(
    x=precipitation_data2['referenceTime'],
    y=precipitation_data2['value'],
    mode='lines',
    name=f'Precipitation at {station_names.get(used_station_id2, "Unknown Station 2")}',
    line=dict(color=color_location2)
))

# Gjennomsnittsnedbør for lokasjon 1
mean_precip1 = precipitation_data1['value'].mean()
fig_precip.add_trace(go.Scatter(
    x=precipitation_data1['referenceTime'],
    y=[mean_precip1] * len(precipitation_data1['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location1),
    name=f'Average Precipitation at {station_names.get(used_station_id1, "Unknown Station 1")}'
))

# Gjennomsnittsnedbør for lokasjon 2
mean_precip2 = precipitation_data2['value'].mean()
fig_precip.add_trace(go.Scatter(
    x=precipitation_data2['referenceTime'],
    y=[mean_precip2] * len(precipitation_data2['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location2),
    name=f'Average Precipitation at {station_names.get(used_station_id2, "Unknown Station 2")}'
))

# Tilpasser layout for nedbør
fig_precip.update_layout(
    title='Nedbør over tid',
    xaxis_title='Dato',
    yaxis_title='Nedbør (mm)',
    legend_title='Lokasjoner'
)

# Viser nedbørsfigur
fig_precip.show()

# Vindhastighetsfigur
fig_wind = go.Figure()

# Vindhastighetsdata for lokasjon 1
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data1['referenceTime'],
    y=wind_speed_data1['value'],
    mode='lines',
    name=f'Wind Speed at {station_names.get(used_station_id1, "Unknown Station 1")}',
    line=dict(color=color_location1)
))

# Vindhastighetsdata for lokasjon 2
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data2['referenceTime'],
    y=wind_speed_data2['value'],
    mode='lines',
    name=f'Wind Speed at {station_names.get(used_station_id2, "Unknown Station 2")}',
    line=dict(color=color_location2)
))

# Gjennomsnittsvindhastighet for lokasjon 1
mean_wind1 = wind_speed_data1['value'].mean()
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data1['referenceTime'],
    y=[mean_wind1] * len(wind_speed_data1['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location1),
    name=f'Average Wind Speed at {station_names.get(used_station_id1, "Unknown Station 1")}'
))

# Gjennomsnittsvindhastighet for lokasjon 2
mean_wind2 = wind_speed_data2['value'].mean()
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data2['referenceTime'],
    y=[mean_wind2] * len(wind_speed_data2['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location2),
    name=f'Average Wind Speed at {station_names.get(used_station_id2, "Unknown Station 2")}'
))

# Tilpasser layout for vindhastighet
fig_wind.update_layout(
    title='Vindhastighet over tid',
    xaxis_title='Dato',
    yaxis_title='Vindhastighet (m/s)',
    legend_title='Lokasjoner'
)

# Viser vindhastighetsfigur
fig_wind.show()